In [1]:
# Range dependent Hamiltonian Algorithm for numerical QUBO formulation
# 1 qubit examples

import numpy as np
import random, math
import copy, sys

Dimension = 2
qubits = 1;
Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

Matrix A:
[[-36  84]
 [ 71 -31]]

Solution vector x:
[1 1]

Vector b:
[48 40]


In [2]:
QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[  4113. -10450.]
 [     0.   2433.]]

Minimum energy is  -3904


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):4113.0, ('q2','q2'):2433.0}

quadratic = {('q1','q2'):-10450.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [3]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):4113.0, ('q2','q2'):2433.0}

quadratic = {('q1','q2'):-10450.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  1  1 -3904.0     999     0.0
1  0  0     0.0       1     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [6]:
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

Extended solution vector x:
[970501 184045]

Vector b:
[-19478256  63200176]


In [7]:
# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

Translation vector T:
[970500. 184044.]

Qubit notation:
[1. 1.]

Updated vector c:
[48. 40.]


In [8]:
QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")  

# Matrix Q is
[[  4113. -10450.]
 [     0.   2433.]]

Minimum energy is  -3904.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):4113.0, ('q2','q2'):2433.0}

quadratic = {('q1','q2'):-10450.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [9]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):4113.0, ('q2','q2'):2433.0}

quadratic = {('q1','q2'):-10450.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  1  1 -3904.0     998     0.0
1  0  0     0.0       2     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [10]:
# 2nd step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

Matrix A:
[[-23 -58]
 [-73  28]]

Solution vector x:
[1 0]

Vector b:
[-23 -73]
# Matrix Q is
[[-5858. -1420.]
 [    0.  5568.]]

Minimum energy is  -5858


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-5858.0, ('q2','q2'):5568.0}

quadratic = {('q1','q2'):-1420.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [11]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-5858.0, ('q2','q2'):5568.0}

quadratic = {('q1','q2'):-1420.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  1  0 -5858.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 2 variables]


In [13]:
# 2nd step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")  

Extended solution vector x:
[-579514  220166]

Vector b:
[  559194 48469170]
Translation vector T:
[-579514.  220166.]

Qubit notation:
[0. 0.]

Updated vector c:
[0. 0.]
# Matrix Q is
[[ 5858. -1420.]
 [    0.  4148.]]

Minimum energy is  -0.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):5858.0, ('q2','q2'):4148.0}

quadratic = {('q1','q2'):-1420.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [14]:
linear = {('q1','q1'):5858.0, ('q2','q2'):4148.0}

quadratic = {('q1','q2'):-1420.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 energy num_oc. chain_.
0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 2 variables]


In [15]:
# 3rd step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

Matrix A:
[[-20  35]
 [-99  37]]

Solution vector x:
[1 1]

Vector b:
[ 15 -62]
# Matrix Q is
[[-1475. -8726.]
 [    0.  6132.]]

Minimum energy is  -4069


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-1475.0, ('q2','q2'):6132.0}

quadratic = {('q1','q2'):-8726.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [16]:
linear = {('q1','q1'):-1475.0, ('q2','q2'):6132.0}

quadratic = {('q1','q2'):-8726.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  1  1 -4069.0     999     0.0
1  1  0 -1475.0       1     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [17]:
# 3rd step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")  

Extended solution vector x:
[654652  14941]

Vector b:
[-12570105 -64257731]
Translation vector T:
[654652.  14940.]

Qubit notation:
[0. 1.]

Updated vector c:
[35. 37.]
# Matrix Q is
[[18927. -8726.]
 [    0. -2594.]]

Minimum energy is  -2594.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):18927.0, ('q2','q2'):-2594.0}

quadratic = {('q1','q2'):-8726.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [18]:
linear = {('q1','q1'):18927.0, ('q2','q2'):-2594.0}

quadratic = {('q1','q2'):-8726.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  0  1 -2594.0     998     0.0
1  0  0     0.0       2     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [19]:
# 4th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[ 58  -1]
 [ 54 -28]]

Solution vector x:
[1 1]

Vector b:
[57 26]
# Matrix Q is
[[-3140. -3140.]
 [    0.  2355.]]

Minimum energy is  -3925


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-3140.0, ('q2','q2'):2355.0}

quadratic = {('q1','q2'):-3140.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [20]:
linear = {('q1','q1'):-3140.0, ('q2','q2'):2355.0}

quadratic = {('q1','q2'):-3140.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  1  1 -3925.0     998     0.0
1  1  0 -3140.0       2     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [21]:
# 4th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[-863735 -835398]

Vector b:
[-49261232 -23250546]
Translation vector T:
[-863736. -835398.]

Qubit notation:
[1. 0.]

Updated vector c:
[58. 54.]
# Matrix Q is
[[-6280. -3140.]
 [    0.  3925.]]

Minimum energy is  -6280.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-6280.0, ('q2','q2'):3925.0}

quadratic = {('q1','q2'):-3140.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [22]:
linear = {('q1','q1'):-6280.0, ('q2','q2'):3925.0}

quadratic = {('q1','q2'):-3140.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  1  0 -6280.0     977     0.0
1  1  1 -5495.0      23     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [23]:
# 5th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[26 53]
 [62 57]]

Solution vector x:
[1 1]

Vector b:
[ 79 119]
# Matrix Q is
[[-14344.   9824.]
 [     0. -15882.]]

Minimum energy is  -20402


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-14344.0, ('q2','q2'):-15882.0}

quadratic = {('q1','q2'):9824.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [24]:
linear = {('q1','q1'):-14344.0, ('q2','q2'):-15882.0}

quadratic = {('q1','q2'):9824.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2   energy num_oc. chain_.
0  1  1 -20402.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 2 variables]


In [25]:
# 5th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[ 756933 -527959]

Vector b:
[-8301569 16836183]
Translation vector T:
[ 756932. -527960.]

Qubit notation:
[1. 1.]

Updated vector c:
[ 79. 119.]
# Matrix Q is
[[-14344.   9824.]
 [     0. -15882.]]

Minimum energy is  -20402.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-14344.0, ('q2','q2'):-15882.0}

quadratic = {('q1','q2'):9824.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [26]:
linear = {('q1','q1'):-14344.0, ('q2','q2'):-15882.0}

quadratic = {('q1','q2'):9824.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2   energy num_oc. chain_.
0  1  1 -20402.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 2 variables]


In [27]:
# 6th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[ 1 48]
 [78 -9]]

Solution vector x:
[1 1]

Vector b:
[49 69]
# Matrix Q is
[[-4777. -1308.]
 [    0. -1077.]]

Minimum energy is  -7162


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-4777.0, ('q2','q2'):-1077.0}

quadratic = {('q1','q2'):-1308.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [28]:
linear = {('q1','q1'):-4777.0, ('q2','q2'):-1077.0}

quadratic = {('q1','q2'):-1308.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  1  1 -7162.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 2 variables]


In [29]:
# 6th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[-400896 -513511]

Vector b:
[-25049424 -26648289]
Translation vector T:
[-400896. -513512.]

Qubit notation:
[0. 1.]

Updated vector c:
[48. -9.]
# Matrix Q is
[[ 7393. -1308.]
 [    0. -2385.]]

Minimum energy is  -2385.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):7393.0, ('q2','q2'):-2385.0}

quadratic = {('q1','q2'):-1308.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [30]:
linear = {('q1','q1'):7393.0, ('q2','q2'):-2385.0}

quadratic = {('q1','q2'):-1308.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  0  1 -2385.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 2 variables]


In [31]:
# 7th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[ -1 -89]
 [-24  -1]]

Solution vector x:
[1 0]

Vector b:
[ -1 -24]
# Matrix Q is
[[-577.  226.]
 [   0. 7696.]]

Minimum energy is  -577


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-577.0, ('q2','q2'):7696.0}

quadratic = {('q1','q2'):226.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [32]:
linear = {('q1','q1'):-577.0, ('q2','q2'):7696.0}

quadratic = {('q1','q2'):226.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 energy num_oc. chain_.
0  1  0 -577.0     945     0.0
1  0  0    0.0      55     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [33]:
# 7th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[-993154 -860115]

Vector b:
[77543389 24695811]
Translation vector T:
[-993154. -860116.]

Qubit notation:
[0. 1.]

Updated vector c:
[-89.  -1.]
# Matrix Q is
[[  351.   226.]
 [    0. -7922.]]

Minimum energy is  -7922.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):351.0, ('q2','q2'):-7922.0}

quadratic = {('q1','q2'):226.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [34]:
linear = {('q1','q1'):351.0, ('q2','q2'):-7922.0}

quadratic = {('q1','q2'):226.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  0  1 -7922.0     947     0.0
1  1  1 -7345.0      53     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [35]:
# 8th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[-30  60]
 [ 87 -65]]

Solution vector x:
[1 0]

Vector b:
[-30  87]
# Matrix Q is
[[ -8469. -14910.]
 [     0.  22735.]]

Minimum energy is  -8469


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-8469.0, ('q2','q2'):22735.0}

quadratic = {('q1','q2'):-14910.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [36]:
linear = {('q1','q1'):-8469.0, ('q2','q2'):22735.0}

quadratic = {('q1','q2'):-14910.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  1  0 -8469.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 2 variables]


In [37]:
# 8th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[288893 780064]

Vector b:
[ 38137050 -25570469]
Translation vector T:
[288892. 780064.]

Qubit notation:
[1. 0.]

Updated vector c:
[-30.  87.]
# Matrix Q is
[[ -8469. -14910.]
 [     0.  22735.]]

Minimum energy is  -8469.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-8469.0, ('q2','q2'):22735.0}

quadratic = {('q1','q2'):-14910.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [38]:
linear = {('q1','q1'):-8469.0, ('q2','q2'):22735.0}

quadratic = {('q1','q2'):-14910.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  1  0 -8469.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 2 variables]


In [39]:
# 9th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[-53 -63]
 [ 47  99]]

Solution vector x:
[1 0]

Vector b:
[-53  47]
# Matrix Q is
[[-5018. 15984.]
 [    0. -2214.]]

Minimum energy is  -5018


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-5018.0, ('q2','q2'):-2214.0}

quadratic = {('q1','q2'):15984.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [40]:
linear = {('q1','q1'):-5018.0, ('q2','q2'):-2214.0}

quadratic = {('q1','q2'):15984.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2  energy num_oc. chain_.
0  1  0 -5018.0     989     0.0
1  0  1 -2214.0      11     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [41]:
# 9th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[124970  52107]

Vector b:
[-9906151 11032183]
Translation vector T:
[124970.  52106.]

Qubit notation:
[0. 1.]

Updated vector c:
[-63.  99.]
# Matrix Q is
[[-10966.  15984.]
 [     0. -13770.]]

Minimum energy is  -13770.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-10966.0, ('q2','q2'):-13770.0}

quadratic = {('q1','q2'):15984.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [42]:
linear = {('q1','q1'):-10966.0, ('q2','q2'):-13770.0}

quadratic = {('q1','q2'):15984.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2   energy num_oc. chain_.
0  0  1 -13770.0     974     0.0
1  1  0 -10966.0      26     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [43]:
# 10th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[ 92 -46]
 [ 86  29]]

Solution vector x:
[1 0]

Vector b:
[92 86]
# Matrix Q is
[[-15860.  -3476.]
 [     0.   6433.]]

Minimum energy is  -15860


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-15860.0, ('q2','q2'):6433.0}

quadratic = {('q1','q2'):-3476.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [44]:
linear = {('q1','q1'):-15860.0, ('q2','q2'):6433.0}

quadratic = {('q1','q2'):-3476.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2   energy num_oc. chain_.
0  1  0 -15860.0     994     0.0
1  1  1 -12903.0       6     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]


In [45]:
# 10th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[-351161  493543]

Vector b:
[-55009790 -15887099]
Translation vector T:
[-351162.  493542.]

Qubit notation:
[1. 1.]

Updated vector c:
[ 46. 115.]
# Matrix Q is
[[-12384.  -3476.]
 [     0.    519.]]

Minimum energy is  -15341.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-12384.0, ('q2','q2'):519.0}

quadratic = {('q1','q2'):-3476.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [46]:
linear = {('q1','q1'):-12384.0, ('q2','q2'):519.0}

quadratic = {('q1','q2'):-3476.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2   energy num_oc. chain_.
0  1  1 -15341.0     999     0.0
1  1  0 -12384.0       1     0.0
['BINARY', 2 rows, 1000 samples, 2 variables]
